# 1. CUDA 설정 확인

In [9]:
!nvidia-smi

Mon Oct 28 21:14:49 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   55C    P8              8W /   35W |    2139MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
#cuda 사용 여부 확인
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
use_cuda = torch.cuda.is_available()
print(use_cuda)
if use_cuda:
  print(torch.cuda.get_device_name(0))

cuda:0
True
NVIDIA GeForce GTX 1650 with Max-Q Design


In [11]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # CUDA 사용 가능 여부
print(torch.version.cuda)          # 현재 설치된 CUDA 버전

2.5.0+cu124
True
12.4


# 2. 모델 테스트

In [12]:
#예상 결과 출력해주는 라이브러리

import cv2
import supervision as sv
import os
from autodistill_grounding_dino import GroundingDINO
from autodistill.detection import CaptionOntology
from groundingdino.util.inference import load_model, load_image, predict, annotate

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'

## 2-1. YOLO-NAS

In [13]:
#from autodistill_yolonas import YOLONAS
#super_gradient 사용불가 문제로 ...

target_model = torch.load("yolo_nas_s.pt", weights_only=False)

img = cv2.imread("./Data/india.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)

result = target_model.predict(img, conf=0.35)
result.show(box_thickness=2, show_confidence=True)

RuntimeError: CUDA error: an illegal memory access was encountered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## 2-2. Grounding DINO

In [6]:
# 이미지 호출
img = cv2.imread("./Data/india.jpg")
img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
#annotated_frame = annotate(image_source=img, boxes=boxes, logits=logits, phrases=phrases)

#오토디스틸에선 이렇게 작동됨
base_model = GroundingDINO(ontology=CaptionOntology({"person": "person"}))

predictions = base_model.predict(img)

annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = annotator.annotate(scene=img, detections=predictions)
labels = []
for i in range(len(predictions)) :
  labels.append("person")

annotated_image = label_annotator.annotate(scene=annotated_image, detections=predictions, labels=labels)

sv.plot_image(img)

trying to load grounding dino directly


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)


final text_encoder_type: bert-base-uncased


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
The `device` argument is deprecated and will be removed in v5 of Transformers.
torch.utils.checkpoint: the use_reentrant parameter should be pas

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 2048 n 20906 k 256 mat1_ld 256 mat2_ld 256 result_ld 2048 abcType 0 computeType 68 scaleType 0

# 3. autodistill로 YOLO-NAS 훈련

## 3-1. raw video에서 이미지 추출

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
#데이터셋 경로지정
import os
HOME = os.getcwd()
print(HOME)

VIDEO_DIR_PATH = f"{HOME}/Data/MiddleTest"
IMAGE_DIR_PATH = f"{HOME}/Data/Silced"
FRAME_STRIDE = 10

In [ ]:
video_paths = sv.list_files_with_extensions(
    directory=VIDEO_DIR_PATH,
    extensions=["mp4"])

TEST_VIDEO_PATHS, TRAIN_VIDEO_PATHS = video_paths[:2], video_paths[2:]

for video_path in tqdm(TRAIN_VIDEO_PATHS):
    video_name = video_path.stem
    image_name_pattern = video_name + "-{:05d}.png"
    with sv.ImageSink(target_dir_path=IMAGE_DIR_PATH, image_name_pattern=image_name_pattern) as sink:
        for image in sv.get_video_frames_generator(source_path=str(video_path), stride=FRAME_STRIDE):
            sink.save_image(image=image)

In [ ]:
import supervision as sv

image_paths = sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["png"])

print('image count:', len(image_paths))

## 3-2. GroundingDINO로 오토라벨링

In [ ]:
#온톨로지 설정... 찾아야 하는거 : 라벨 형식
#https://docs.autodistill.com/quickstart/
from autodistill.detection import CaptionOntology

#분량이 너무 많아서 중간 발표 전에 테스트 불가능
'''
ontology=CaptionOntology({
    "person walking with dog": "개와 산책하는 사람",
    "person riding bicycle": "자전거 타는 사람",
    "person wearing a hat": "모자 쓴 사람",
    "person with umbrella": "우산 쓴 사람",
    "person holding a phone": "전화하는 사람",
    "blue signs": "파란 표지판",
    "yellow signs": "노란 표지판",
    "truck": "트럭",
    "car": "자동차",
    "helicopter": "헬리콥터",
    "blue bus": "파란색 버스",
    "green bus": "녹색 버스",
    "red bus": "빨간색 버스",
    "bird": "새",
    "yellow cat": "노란 고양이",
    "black cat": "검정 고양이",
    "tree": "나무",
    "flower": "꽃",
    "fire hydrant": "소화전",
    "electric scooter": "전동스쿠터",
    "bicycle": "자전거",
    "ladder": "사다리",
    "manhole": "맨홀",
    "traffic Cone": "라바콘"
})
'''

#중간 테스트 용도
ontology=CaptionOntology({
    "helicopter": "헬리콥터",
    "person riding bicycle": "자전거 타는 사람",
    "electric scooter": "전동스쿠터",
    "traffic Cone": "라바콘"
})

In [ ]:
#자동 라벨링, 데이터셋 제작

DATASET_DIR_PATH = f"{HOME}/Data/Labeled"
base_model = GroundingDINO(ontology=ontology)
dataset = base_model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".png",
    output_folder=DATASET_DIR_PATH)

## 3-3. YOLO-NAS-s로 결과물 학습

In [ ]:
# Step #5: Train a YOLO-NAS Model
from autodistill_yolonas import YOLONAS

# 원래대로라면 모델명 기입해야 함.
target_model = YOLONAS("")

# 내부에서 torch.load("/content/sample_data/yolo_nas_s.pt") 실행 후 모델 훈련.
target_model.train("./경로", epochs=20)

# run inference on the new model
pred = target_model.predict("./경로", confidence=0.5)